In [1]:
import cv2

img = cv2.imread("../input/satellite-image-classification/data/cloudy/train_10070.jpg")

img.shape


In [2]:
import matplotlib.pyplot as plt

plt.imshow(img)

In [3]:
img_dir = "../input/satellite-image-classification/data"

In [4]:
# processing the dataset

import tensorflow as tf 

train_data = tf.keras.preprocessing.image_dataset_from_directory(img_dir,
                                                                 image_size = (64,64),
                                                                 label_mode = "categorical",
                                                                 batch_size= 32,
                                                                 validation_split = 0.20,
                                                                 seed = 42,
                                                                 subset = "training"
                                                                 )

test_data = tf.keras.preprocessing.image_dataset_from_directory(img_dir,
                                                                image_size = (64,64),
                                                                label_mode = "categorical",
                                                                batch_size = 32,
                                                                seed = 42,
                                                                validation_split = 0.20,
                                                                subset = "validation"
                                                                )

In [5]:
class_labels = train_data.class_names

class_labels

In [6]:
# visualizing the train datset

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(14,14))

for image, label in train_data.take(1):
  
  for i in range(18):
    plt.subplot(6,3,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(class_labels[tf.argmax(label[i])]) 
    plt.axis("off")


In [7]:
# building a model on efficientnet

import tensorflow as tf

base_model = tf.keras.applications.EfficientNetB5(include_top= False)

base_model.trainable = False

In [8]:
import tensorflow as tf
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(64,64,3))

x = base_model(inputs)

x = layers.GlobalAveragePooling2D()(x)

outputs = layers.Dense(4, activation= "softmax")(x)

eff_model = tf.keras.Model(inputs,outputs)


In [9]:
# compiling the model

eff_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [10]:
# fitting data to the model

eff_model.fit(train_data,
              epochs = 5,
              steps_per_epoch = len(train_data),
              validation_data = test_data,
              validation_steps = len(test_data)
              )

In [28]:
# tunning the model

base_model.trainable = True

for layer in base_model.layers[:-3]:
  base_model.trainable = False


eff_model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

eff_model.fit(
    train_data,
    epochs = 5,
    steps_per_epoch = len(train_data),
    validation_data = test_data,
    validation_steps = len(test_data)
)

In [29]:
model_evaluation = eff_model.evaluate(test_data)

In [30]:
print(f"Model Accuarcy: {model_evaluation[1] * 100 : 0.2f} %")

In [40]:
# visualizing the model prediction 

import tensorflow as tf
import matplotlib.pyplot as plt

plt.figure(figsize=(16,16))

for image, label in test_data.take(1):
  
  model_pred = eff_model.predict(image)

  for i in range(18):

    plt.subplot(6,3,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(f"Prediction: {class_labels[tf.argmax(model_pred[i])]} \nOriginal : {class_labels[tf.argmax(label[i])]}")
    plt.subplots_adjust(top = 1.25)
    plt.axis("off")
